In [ ]:
NAME = "Brandon Ng"

---

# Sudoku Solver

First, I define a representation for a Sudoku problem. 

The idea is to represent a Sudoku problem via a $9 \times 9$ matrix of _sets_: each set contains the digits that can possibly fit in the given space.   A known digit will be represented with just a set containing only one element or 0 to signify "blank".
The code will solve a Sudoku problem by progressively "shrinking" these sets of possibilities, until they all contain exactly one element. 


Tiny helper function that returns the only element from a singleton set.

In [ ]:
def getel(s):
    """Returns the unique element in a singleton set (or list)."""
    assert len(s) == 1
    return list(s)[0]


In [ ]:
import json

class Sudoku(object):

    def __init__(self, elements):
        """Elements can be one of:
        Case 1: a list of 9 strings of length 9 each.
        Each string represents a row of the initial Sudoku puzzle,
        with either a digit 1..9 in it, or with a blank or _ to signify
        a blank cell.
        Case 2: an instance of Sudoku.  In that case, we initialize an
        object to be equal (a copy) of the one in elements.
        Case 3: a list of list of sets, used to initialize the problem."""
        if isinstance(elements, Sudoku):
            # We let self.m consist of copies of each set in elements.m
            self.m = [[x.copy() for x in row] for row in elements.m]
        else:
            assert len(elements) == 9
            for s in elements:
                assert len(s) == 9
            # We let self.m be our Sudoku problem, a 9x9 matrix of sets.
            self.m = []
            for s in elements:
                row = []
                for c in s:
                    if isinstance(c, str):
                        if c.isdigit():
                            row.append({int(c)})
                        else:
                            row.append({1, 2, 3, 4, 5, 6, 7, 8, 9})
                    else:
                        assert isinstance(c, set)
                        row.append(c)
                self.m.append(row)


    def show(self, details=False):
        """Prints out the Sudoku matrix.  If details=False, we print out
        the digits only for cells that have singleton sets (where only
        one digit can fit).  If details=True, for each cell, we display the
        sets associated with the cell."""
        if details:
            print("+-----------------------------+-----------------------------+-----------------------------+")
            for i in range(9):
                r = '|'
                for j in range(9):
                    # We represent the set {2, 3, 5} via _23_5____
                    s = ''
                    for k in range(1, 10):
                        s += str(k) if k in self.m[i][j] else '_'
                    r += s
                    r += '|' if (j + 1) % 3 == 0 else ' '
                print(r)
                if (i + 1) % 3 == 0:
                    print("+-----------------------------+-----------------------------+-----------------------------+")
        else:
            print("+---+---+---+")
            for i in range(9):
                r = '|'
                for j in range(9):
                    if len(self.m[i][j]) == 1:
                        r += str(getel(self.m[i][j]))
                    else:
                        r += "."
                    if (j + 1) % 3 == 0:
                        r += "|"
                print(r)
                if (i + 1) % 3 == 0:
                    print("+---+---+---+")


    def to_string(self):
        """This method is useful for producing a representation that
        can be used in testing."""
        as_lists = [[list(self.m[i][j]) for j in range(9)] for i in range(9)]
        return json.dumps(as_lists)


    @staticmethod
    def from_string(s):
        """Inverse of above."""
        as_lists = json.loads(s)
        as_sets = [[set(el) for el in row] for row in as_lists]
        return Sudoku(as_sets)


    def __eq__(self, other):
        """Useful for testing."""
        return self.m == other.m


Input a problem page and check that the serialization and deserialization works.

In [ ]:
# Ensure that nose is installed.
try:
    from nose.tools import assert_equal, assert_true
    from nose.tools import assert_false, assert_almost_equal
except:
    !pip install nose
    from nose.tools import assert_equal, assert_true
    from nose.tools import assert_false, assert_almost_equal


     |████████████████████████████████| 154 kB 4.4 MB/s 


In [ ]:
from nose.tools import assert_equal

sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.show()
sd.show(details=True)
s = sd.to_string()
sdd = Sudoku.from_string(s)
sdd.show(details=True)
assert_equal(sd, sdd)


+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|419|..5|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 123456789|123456789 ______7__ 123456789|123456789 123456789 123456789|
|_____6___ 123456789 123456789|1________ ________9 ____5____|123456789 123456789 123456789|
|123456789 ________9 _______8_|123456789 123456789 123456789|123456789 _____6___ 123456789|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 123456789 123456789|123456789 _____6___ 123456789|123456789 123456789 __3______|
|___4_____ 123456789 123456789|_______8_ 123456789 __3______|123456789 123456789 1________|
|______7__ 123456789 123456789|123456789 _2_______ 123456789|123456789 123456789 _____6___|
+-----------------------------+-----------------------------+---------------------

Test the constructor statement when passed a Sudoku instance.

In [ ]:
sd1 = Sudoku(sd)
assert_equal(sd, sd1)


## Constraint propagation

When the set in a Sudoku cell contains only one element, this means that the digit at that cell is known. 
We can then propagate the knowledge, ruling out that digit in the same row, in the same column, and in the same 3x3 cell. 

I first write a method that propagates the constraint from a single cell.  The method will return the list of newly-determined cells, that is, the list of cells who also now (but not before) are associated with a 1-element set.  This is useful, because we can then propagate the constraints from those cells in turn.  Further, if an empty set is ever generated, we raise the exception Unsolvable: this means that there is no solution to the proposed Sudoku puzzle. 

## Part 1: Propagating a single cell

In [ ]:
class Unsolvable(Exception):
    pass


def sudoku_ruleout(self, i, j, x):
    """The input consists in a cell (i, j), and a value x.
    The function removes x from the set self.m[i][j] at the cell, if present, and:
    - if the result is empty, raises Unsolvable;
    - if the cell used to be a non-singleton cell and is now a singleton
      cell, then returns the set {(i, j)};
    - otherwise, returns the empty set."""
    c = self.m[i][j]
    n = len(c)
    c.discard(x)
    self.m[i][j] = c
    if len(c) == 0:
        raise Unsolvable()
    return {(i, j)} if 1 == len(c) < n else set()

Sudoku.ruleout = sudoku_ruleout


The method `propagate_cell(ij)` takes as input a pair `ij` of coordinates.  If the set of possible digits `self.m[i][j]` for cell i,j contains more than one digit, then no propagation is done.  If the set contains a single digit `x`, then we: 

* Remove `x` from the sets of all other cells on the same row, column, and 3x3 block. 
* Collect all the newly singleton cells that are formed, due to the digit `x` being removed, and we return them as a set. 

In [ ]:
### Exercise: define cell propagation

def sudoku_propagate_cell(self, ij):
    """Propagates the singleton value at cell (i, j), returning the list
    of newly-singleton cells."""
    i, j = ij
    if len(self.m[i][j]) > 1:
        # Nothing to propagate from cell (i,j).
        return set()
    # We keep track of the newly-singleton cells.
    newly_singleton = set()
    x = getel(self.m[i][j]) # Value at (i, j).
    # Same row.
    for jj in range(9):
        if jj != j: # Do not propagate to the element itself.
            newly_singleton.update(self.ruleout(i, jj, x))
    # Same column.
    for ii in range(9):
        if ii != i:
            newly_singleton.update(self.ruleout(ii, j, x))
    # Same block of 3x3 cells.
    for ii in range(i//3*3, i//3*3+3):
        for jj in range(j//3*3, j//3*3+3):
            if ii != i and jj != j:
                newly_singleton.update(self.ruleout(ii,jj, x))
    # Returns the list of newly-singleton cells.
    return newly_singleton

Sudoku.propagate_cell = sudoku_propagate_cell


In [ ]:
tsd = Sudoku.from_string('[[[5], [3], [2], [6], [7], [8], [9], [1, 2, 4], [2]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
tsd.show(details=True)
try:
    tsd.propagate_cell((0, 2))
except Unsolvable:
    print("Good! It was unsolvable.")
else:
    raise Exception("Hey, it was unsolvable")

tsd = Sudoku.from_string('[[[5], [3], [2], [6], [7], [8], [9], [1, 2, 4], [2, 3]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
tsd.show(details=True)
assert_equal(tsd.propagate_cell((0, 2)), {(0, 8), (2, 0)})



+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ _2_______|_____6___ ______7__ _______8_|________9 12_4_____ _2_______|
|_____6___ ______7__ 12_4__7__|123______ ________9 ____5____|__3______ 12_4_____ _______8_|
|12_______ ________9 _______8_|__3______ ___4_____ 12_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ ____5____ ________9|1_____7_9 _____6___ 1__4__7_9|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1________ __3______|________9 _2_______ ___4_____|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 _______8_ _2____7_9|___4_____ 1________ _2____7_9|_____6___ __3______

### Propagating all cells, once

In [ ]:
def sudoku_propagate_all_cells_once(self):
    """This function propagates the constraints from all singletons."""
    for i in range(9):
        for j in range(9):
            self.propagate_cell((i, j))

Sudoku.propagate_all_cells_once = sudoku_propagate_all_cells_once


In [ ]:
sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.show()
sd.propagate_all_cells_once()
sd.show()
sd.show(details=True)


+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|419|..5|
|...|.8.|.79|
+---+---+---+
+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|853|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|..7|284|
|...|419|.35|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|_2___6___ ______7__ _2_4_6_8_|1__4___89 12_4____9 _2_4___8_|
|_____6___ _2_4__7__ _2_4__7__|1________ ________9 ____5____|__34__78_ _234_____ _2_4__78_|
|12_______ ________9 _______8_|_23______ __34_____ _2_4_____|1_345_7__ _____6___ _2_4__7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 12__5____ 12__5___9|____5_7_9 _____6___ 1__4__7__|___45_7_9 _2_45___9 __3______|
|___4_____ _2__5____ _2__56__9|_______8_ ____5____ __3______|____5_7_9 _2__5___9 1__

## Part 2: Propagating all cells, repeatedly


The method `full_propagation` starts with a set of `to_propagate` cells (if it is not specified, then we just take it to consist of all singleton cells).  Then, it picks a cell from the to_propagate set, and propagates from it, adding any newly singleton cell to to_propagate.  Once there are no more cells to be propagated, the method returns. 
It is the algorithmic pattern of keeping a list of work to be done, then iteratively picking an element from the list, doing it, possibly updating the list of work to be done with new work that has to be done as a result of what we just did, and continuing in this fashion until there is nothing left to do. 

In [ ]:
def sudoku_full_propagation(self, to_propagate=None):
    """Iteratively propagates from all singleton cells, and from all
    newly discovered singleton cells, until no more propagation is possible.
    @param to_propagate: sets of cells from where to propagate.  If None, propagates
        from all singleton cells. 
    @return: nothing.
    """
    if to_propagate is None:
        to_propagate = {(i, j) for i in range(9) for j in range(9)}
    # This code is the (A) code; will be referenced later.
    queue = to_propagate
    while queue != set():
        val = self.propagate_cell(to_propagate.pop())
        to_propagate.update(val)
        

Sudoku.full_propagation = sudoku_full_propagation


In [ ]:
# Tests for full propagation

sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.full_propagation()
sd.show(details=True)
sdd = Sudoku.from_string('[[[5], [3], [4], [6], [7], [8], [9], [1], [2]], [[6], [7], [2], [1], [9], [5], [3], [4], [8]], [[1], [9], [8], [3], [4], [2], [5], [6], [7]], [[8], [5], [9], [7], [6], [1], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[9], [6], [1], [5], [3], [7], [2], [8], [4]], [[2], [8], [7], [4], [1], [9], [6], [3], [5]], [[3], [4], [5], [2], [8], [6], [1], [7], [9]]]')
assert_equal(sd, sdd)



+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ ___4_____|_____6___ ______7__ _______8_|________9 1________ _2_______|
|_____6___ ______7__ _2_______|1________ ________9 ____5____|__3______ ___4_____ _______8_|
|1________ ________9 _______8_|__3______ ___4_____ _2_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ ____5____ ________9|______7__ _____6___ 1________|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1________ __3______|________9 _2_______ ___4_____|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|________9 _____6___ 1________|____5____ __3______ ______7__|_2_______ _______8_ ___4_____|
|_2_______ _______8_ ______7__|___4_____ 1________ ________9|_____6___ __3______

## Searching for a solution

Many Sudoku problems can be solved entirely by constraint propagation.  
 

But it is by no means necessary that this is true. 
If we create more complex problems, or less determined problems, constraint propagation no longer suffices. 

In [ ]:
#Example of constraint propagation not solving the Sudoku
sd = Sudoku([
    '53__7____',
    '6___95___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___41___5',
    '____8__79'
])
sd.show()
sd.full_propagation()
sd.show(details=True)


+---+---+---+
|53.|.7.|...|
|6..|.95|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|41.|..5|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ 12___6_8_|___4___89 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|__3_5____ _____6___ _2____7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_45____ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ 1_3_5___9|1_______9 _2_______ 1__4____9|___45__8_ ___45____ _____6___|
+-----------------------------+-----------------------------+---------------------

In [ ]:
sd.show(details=True)
sd_partially_solved = Sudoku(sd)


+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ 12___6_8_|___4___89 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|__3_5____ _____6___ _2____7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_45____ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ 1_3_5___9|1_______9 _2_______ 1__4____9|___45__8_ ___45____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 ______78_ _2____7_9|___4_____ 1________ _2____7_9|_____6___ __3______

Implement search with backtracking.  The idea works something like this:

search():
1. propagate constraints.
1. if solved, hoorrayy!
1. if impossible, raise Unsolvable()
1. if not fully solved, pick a cell with multiple digits possible, and iteratively:
 * Assign one of the possible values to the cell. 
 * Call search() with that value for the cell.
 * If Unsolvable is raised by the search() call, move on to the next value.
 * If all values returned Unsolvable (if we tried them all), then we raise Unsolvable.

One fine point with the search above is that an object has a self.m matrix, which contains the status of the Sudoku solution. 
You cannot  pass self.m recursively to search(), because in the course of the search and constraint propagation, self.m will be modified, and there is no easy way to keep track of these modifications. 
Rather, I wrote search() as a method, and when I call it, I:

* First, create a copy of the current object via the Sudoku constructor, so I have a copy I can modify. 
* Second, I assign one of the values to the cell, as above; 
* Third, I call the search() method of that object. 

Furthermore, when a solution is found I return the solution via standard returns, or by raising an exception. 


In [ ]:
def sudoku_done(self):
    """Checks whether an instance of Sudoku is solved."""
    for i in range(9):
        for j in range(9):
            if len(self.m[i][j]) > 1:
                return False
    return True

Sudoku.done = sudoku_done


def sudoku_search(self, new_cell=None):
    """Tries to solve a Sudoku instance."""
    to_propagate = None if new_cell is None else {new_cell}
    self.full_propagation(to_propagate=to_propagate)
    if self.done():
        return self # We are a solution
    # We need to search.  Picks a cell with as few candidates as possible.
    candidates = [(len(self.m[i][j]), i, j)
                   for i in range(9) for j in range(9) if len(self.m[i][j]) > 1]
    _, i, j = min(candidates)
    values = self.m[i][j]
    # values contains the list of values we need to try for cell i, j.
    # print("Searching values", values, "for cell", i, j)
    for x in values:
        # print("Trying value", x)
        sd = Sudoku(self)
        sd.m[i][j] = {x}
        try:
            # If we find a solution, we return it.
            return sd.search(new_cell=(i, j))
        except Unsolvable:
            # Go to next value.
            pass
    # All values have been tried, apparently with no success.
    raise Unsolvable()

Sudoku.search = sudoku_search


def sudoku_solve(self, do_print=True):
    """Wrapper function, calls self and shows the solution if any."""
    try:
        r = self.search()
        if do_print:
            print("We found a solution:")
            r.show()
            return r
    except Unsolvable:
        if do_print:
            print("The problem has no solutions")

Sudoku.solve = sudoku_solve


Reexamining previous Sudoku problem that was not solvable via constraint propagation alone.

In [ ]:
sd = Sudoku([
    '53__7____',
    '6___95___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___41___5',
    '____8__79'
])
_ = sd.solve()


We found a solution:
+---+---+---+
|531|678|942|
|674|295|318|
|298|341|567|
+---+---+---+
|859|167|423|
|426|853|791|
|713|924|856|
+---+---+---+
|165|739|284|
|987|412|635|
|342|586|179|
+---+---+---+


## Part 3: A better `full_propagation` method

Before, the code only constraint was the uniqueness in each row, column, and block, the code needed to propagate only from cells that hold a singleton value. 
If a cell held a non-singleton set of digits, such as $\{2, 5\}$, no values could be ruled out as a consequence of this on the same row, column, or block. 

The _where can it go_ heuristic, instead, benefits from knowing that in a cell, the set of values went for instance from $\{2, 3, 5\}$ to $\{2, 5\}$: by ruling out the possibility of a $3$ in this cell, it may be possibe to deduct that the digit $3$ can appear in only one (other) place in the block, and place it there. 

Thus, modifying the `full_propagation` method.  The method does:
* Repeat:
  * first does propagation as before, based on singletons; 
  * then, it applies the _where can it go_ heuristic on the whole Sudoku board. 
* until there is nothing more that can be propagated. 

Thus, I replace the `full_propagation` method previously defined with this new one.

In [ ]:
def sudoku_full_propagation_with_where_can_it_go(self, to_propagate=None):
    """Iteratively propagates from all singleton cells, and from all
    newly discovered singleton cells, until no more propagation is possible."""
    if to_propagate is None:
        to_propagate = {(i, j) for i in range(9) for j in range(9)}
    while len(to_propagate) > 0:
        queue = to_propagate
        while queue != set():
            val = self.propagate_cell(to_propagate.pop())
            to_propagate.update(val)
        # Now we check whether there is any other propagation that we can
        # get from the where can it go rule.
        to_propagate = self.where_can_it_go()



## Part 4: Implement the `occurs_once_in_sets` helper

Given a sequence of sets $S_1, S_2, \ldots, S_n$, we want to obtain the set of elements that appear in _exactly one_ of the sets (that is, they appear in one set, and _only_ in one set).   Mathematically, we can write this as 
$$
(S_1 \setminus (S_2 \cup \cdots \cup S_n)) \cup (S_2 \setminus (S_1 \cup S_3 \cup \cdots \cup S_n)) \cup \cdots \cup
(S_n \setminus (S_1 \cup \cdots \cup S_{n-1}))
$$
even though that's certainly not the easiest way to compute it!
The problem can be solved with the help of [defaultdict](https://docs.python.org/3/library/collections.html#collections.defaultdict!) to count the occurrences.

In [ ]:
from collections import defaultdict

def occurs_once_in_sets(set_sequence):
    """Returns the elements that occur only once in the sequence of sets set_sequence.
    The elements are returned as a set."""
    unique = set()
    d = defaultdict(int)
    for s in set_sequence:
        for e in s:
            d[e] += 1
    for k in d.keys():
        if d.get(k) == 1:
            unique.add(k)
    return unique


In [ ]:
#Tests for once-only

from nose.tools import assert_equal

assert_equal(occurs_once_in_sets([{1, 2}, {2, 3}]), {1, 3})
assert_equal(occurs_once_in_sets([]), set())
assert_equal(occurs_once_in_sets([{2, 3, 4}]), {2, 3, 4})
assert_equal(occurs_once_in_sets([set()]), set())
assert_equal(occurs_once_in_sets([{2, 3, 4, 5, 6}, {5, 6, 7, 8}, {5, 6, 7}, {4, 6, 7}]), {2, 3, 8})



## Part 5: Implement _where can it go_. 

The method is global: it examines all rows, all columns, and all blocks.  
If it finds that in a row (or column, or block), a value can fit in only one cell, and that cell is not currently a singleton (for otherwise there is nothing to be done), it sets the value in the cell, and it adds the cell to the newly_singleton set that is returned, just as in propagate_cell. 

In [ ]:
def sudoku_where_can_it_go(self):
    """Sets some cell values according to the where can it go
    heuristics, by examining all rows, colums, and blocks."""
    newly_singleton = set()

    for i in range(9):
        row = [self.m[i][j] for j in range(9)] 
        value = occurs_once_in_sets(row)
        for v in value:
            for coord in range(9):
                if v in self.m[i][coord] and len(self.m[i][coord]) > 1:
                    self.m[i][coord] = {v}
                    newly_singleton.add((i,coord))

    for j in range(9):
        columns = [self.m[i][j] for i in range(9)] 
        value = occurs_once_in_sets(columns)
        for v in value:
            for coord in range(9):
                if v in self.m[coord][j] and len(self.m[coord][j]) > 1:
                    self.m[coord][j] = {v}
                    newly_singleton.add((coord,j))

    for row in range(0, 9, 3):
        for column in range(0, 9, 3):
            block = [self.m[i][j] for i in range(row, row+3) for j in range(column, column+3)]
            value = occurs_once_in_sets(block) 
            for v in value:
                for coordi in range(row, row+3):
                    for coordj in range(column, column+3):
                        if v in self.m[coordi][coordj] and len(self.m[coordi][coordj]) > 1:
                            self.m[coordi][coordj] = {v}
                            newly_singleton.add((coordi,coordj))

    # Returns the list of newly-singleton cells.
    return newly_singleton

Sudoku.where_can_it_go = sudoku_where_can_it_go


In [ ]:
# Tests for where can it go

sd = Sudoku.from_string('[[[5], [3], [1, 2, 4], [1, 2, 6], [7], [1, 2, 6, 8], [4, 8, 9], [1, 2, 4], [2, 8]], [[6], [1, 4, 7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3, 4, 8], [1, 2, 4], [2, 7, 8]], [[1, 2], [9], [8], [1, 2, 3], [4], [1, 2], [3, 5], [6], [2, 7]], [[8], [1, 5], [1, 5, 9], [1, 7, 9], [6], [1, 4, 7, 9], [4, 5], [2, 4, 5], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1, 5], [1, 3, 5, 9], [1, 9], [2], [1, 4, 9], [4, 5, 8], [4, 5], [6]], [[1, 9], [6], [1, 5, 7, 9], [5, 7, 9], [3], [7, 9], [2], [8], [4]], [[2, 9], [7, 8], [2, 7, 9], [4], [1], [2, 7, 9], [6], [3], [5]], [[2, 3], [4, 5], [2, 3, 4, 5], [2, 5, 6], [8], [2, 6], [1], [7], [9]]]')
print("Original:")
sd.show(details=True)
new_singletons = set()
while True:
    new_s = sd.where_can_it_go()
    if len(new_s) == 0:
        break
    new_singletons |= new_s
assert_equal(new_singletons,
             {(3, 2), (2, 6), (7, 1), (5, 6), (2, 8), (8, 0), (0, 5), (1, 6),
              (2, 3), (3, 7), (0, 3), (5, 1), (0, 8), (8, 5), (5, 3), (5, 5),
              (8, 1), (5, 7), (3, 1), (0, 6), (1, 8), (3, 6), (5, 2), (1, 1)})
print("After where can it go:")
sd.show(details=True)
sdd = Sudoku.from_string('[[[5], [3], [1, 2, 4], [6], [7], [8], [9], [1, 2, 4], [2]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
print("The above should be equal to:")
sdd.show(details=True)
assert_equal(sd, sdd)

sd = Sudoku([
    '___26_7_1',
    '68__7____',
    '1____45__',
    '82_1___4_',
    '__46_2___',
    '_5___3_28',
    '___3___74',
    '_4__5__36',
    '7_3_18___'
])
print("Another Original:")
sd.show(details=True)
print("Propagate once:")
sd.propagate_all_cells_once()
# sd.show(details=True)
new_singletons = set()
while True:
    new_s = sd.where_can_it_go()
    if len(new_s) == 0:
        break
    new_singletons |= new_s
print("After where can it go:")
sd.show(details=True)
sdd = Sudoku.from_string('[[[4], [3], [5], [2], [6], [9], [7], [8], [1]], [[6], [8], [2], [5], [7], [1], [4], [9], [3]], [[1], [9], [7], [8], [3], [4], [5], [6], [2]], [[8], [2], [6], [1], [9], [5], [3], [4], [7]], [[3], [7], [4], [6], [8], [2], [9], [1], [5]], [[9], [5], [1], [7], [4], [3], [6], [2], [8]], [[5], [1], [1, 2, 5, 6, 8, 9], [3], [2], [6], [1, 2, 8, 9], [7], [4]], [[2], [4], [1, 2, 8, 9], [9], [5], [7], [1, 2, 8, 9], [3], [6]], [[7], [6], [3], [4], [1], [8], [2], [5], [9]]]')
print("The above should be equal to:")
sdd.show(details=True)
assert_equal(sd, sdd)



Original:
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|12___6___ ______7__ 12___6_8_|___4___89 12_4_____ _2_____8_|
|_____6___ 1__4__7__ 12_4__7__|123______ ________9 ____5____|__34___8_ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|__3_5____ _____6___ _2____7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ 1___5___9|1_____7_9 _____6___ 1__4__7_9|___45____ _2_45____ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ 1_3_5___9|1_______9 _2_______ 1__4____9|___45__8_ ___45____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 ______78_ _2____7_9|___4_____ 1________ _2____7_9|_____6___

Testing it now on a real probem

In [ ]:
sd = Sudoku(sd_partially_solved)
newly_singleton = sd.where_can_it_go()
print("Newly singleton:", newly_singleton)
print("Resulting Sudoku:")
sd.show(details=True)


Newly singleton: {(3, 2), (2, 6), (7, 1), (5, 6), (2, 8), (8, 0), (5, 7), (0, 6), (0, 5), (1, 6), (3, 6), (3, 7), (0, 3), (5, 2), (1, 1)}
Resulting Sudoku:
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|_____6___ ______7__ _______8_|________9 12_4_____ _2_____8_|
|_____6___ ______7__ 12_4__7__|123______ ________9 ____5____|__3______ 12_4_____ _2____78_|
|12_______ ________9 _______8_|123______ ___4_____ 12_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 1___5____ ________9|1_____7_9 _____6___ 1__4__7_9|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1___5____ __3______|1_______9 _2_______ 1__4____9|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____

The heuristics led to substantial progress

In [ ]:
Sudoku.full_propagation = sudoku_full_propagation_with_where_can_it_go


Trying again to solve the same Sudoku example via constraint propagation?

In [ ]:
sd = Sudoku([
    '53__7____',
    '6___95___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___41___5',
    '____8__79'
])
print("Initial:")
sd.show()
sd.full_propagation()
print("After full propagation with where can it go:")
sd.show()


Initial:
+---+---+---+
|53.|.7.|...|
|6..|.95|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|41.|..5|
|...|.8.|.79|
+---+---+---+
After full propagation with where can it go:
+---+---+---+
|53.|678|9.2|
|67.|.95|3.8|
|.98|34.|567|
+---+---+---+
|859|.6.|423|
|426|853|791|
|713|924|856|
+---+---+---+
|.6.|.3.|284|
|.8.|41.|635|
|34.|.86|179|
+---+---+---+


## Part 6: Solving some problems from example sites

Examines how long it takes for the code to solve examples found around the Web. 
We consider a few from [this site](https://dingo.sbs.arizona.edu/~sandiway/sudoku/examples.html).

In [ ]:
import time


### Daily Telegraph January 19th "Diabolical"



In [ ]:
sd = Sudoku([
    '_2_6_8___',
    '58___97__',
    '____4____',
    '37____5__',
    '6_______4',
    '__8____13',
    '____2____',
    '__98___36',
    '___3_6_9_'
])
t = time.time()
sd.solve()
elapsed = time.time() - t
print("Solved in", elapsed, "seconds")

assert elapsed < 5

We found a solution:
+---+---+---+
|123|678|945|
|584|239|761|
|967|145|328|
+---+---+---+
|372|461|589|
|691|583|274|
|458|792|613|
+---+---+---+
|836|924|157|
|219|857|436|
|745|316|892|
+---+---+---+
Solved in 0.01014089584350586 seconds


### Vegard Hanssen puzzle 2155141

In [ ]:
sd = Sudoku([
    '___6__4__',
    '7____36__',
    '____91_8_',
    '_________',
    '_5_18___3',
    '___3_6_45',
    '_4_2___6_',
    '9_3______',
    '_2____1__'
])
t = time.time()
sd.solve()
elapsed = time.time() - t
print("Solved in", elapsed, "seconds")
assert elapsed < 5

We found a solution:
+---+---+---+
|581|672|439|
|792|843|651|
|364|591|782|
+---+---+---+
|438|957|216|
|256|184|973|
|179|326|845|
+---+---+---+
|845|219|367|
|913|768|524|
|627|435|198|
+---+---+---+
Solved in 0.0469818115234375 seconds


### A supposedly even harder sudoku problem

[source](http://www.sudokuwiki.org/Weekly_Sudoku.asp?puz=28)

In [ ]:
sd = Sudoku([
    '6____894_',
    '9____61__',
    '_7__4____',
    '2__61____',
    '______2__',
    '_89__2___',
    '____6___5',
    '_______3_',
    '8____16__'
])
t = time.time()
sd.solve()
elapsed = time.time() - t
print("Solved in", elapsed, "seconds")
assert elapsed < 10

We found a solution:
+---+---+---+
|625|178|943|
|948|326|157|
|371|945|862|
+---+---+---+
|257|619|384|
|463|587|291|
|189|432|576|
+---+---+---+
|792|863|415|
|516|294|738|
|834|751|629|
+---+---+---+
Solved in 0.6625332832336426 seconds


## Part 7: Trying puzzles in bulk

Trying the puzzles found at [https://raw.githubusercontent.com/shadaj/sudoku/master/sudoku17.txt](https://raw.githubusercontent.com/shadaj/sudoku/master/sudoku17.txt)

In [ ]:
import requests

r = requests.get("https://raw.githubusercontent.com/shadaj/sudoku/master/sudoku17.txt")
puzzles = r.text.split()


In [ ]:
def convert_to_our_format(s):
    t = s.replace('0', '_')
    r = []
    for i in range(9):
        r.append(t[i * 9: (i + 1) * 9])
    return r


Efficiency Test below

In [ ]:
t = 0
max_d = 0.
max_i = None
t = time.time()
for i, s in enumerate(puzzles[:1000]):
    p = convert_to_our_format(puzzles[i])
    sd = Sudoku(p)
    sd.solve(do_print=False)
elapsed = time.time() - t
print("It took you", elapsed, "to solve the first 1000 Sudokus.")
assert elapsed < 30

It took you 12.602380990982056 to solve the first 1000 Sudokus.
